# Naive Bayes 

- 이 노트북에서는 나이브 베이즈를 사용하여 요일, 출발 시간, 출발지, 목적지, 항공사와 같은 다양한 요인을 바탕으로 항공편이 지연될지 여부를 예측하는 분류 모델을 구축합니다.

- 분류 모델의 훈련, 평가 및 예측을 위해 scikit-learn 패키지를 사용합니다. 또한 데이터 조작을 위해 pandas 라이브러리를 활용합니다. 나이브 베이즈 모델에는 scikit-learn의 MultinomialNB 함수를 사용합니다.

## 1. Naive Bayes on "Flight Delays" dataset

### (1) Prepare the data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt

In [ ]:
url = "https://raw.github.com/niharikabalachandra/Logistic-Regression/master/FlightDelays.csv"
delays_df = pd.read_csv(url)

In [ ]:
delays_df.head()

### (2) Preprocessing

- 특정 열을 범주형 변수로 변환합니다.
- 출발 시간을 반올림하여 시간별 구간을 만듭니다.

In [ ]:
# convert to categorical
delays_df.dayweek = delays_df.dayweek.astype('category')
delays_df['delay'] = delays_df['delay'].astype('category')

# create hourly bins departure time 
delays_df.schedtime = [round(t / 100) for t in delays_df.schedtime]
delays_df.schedtime = delays_df.schedtime.astype('category')

predictors = ['dayweek', 'schedtime', 'origin', 'dest', 'carrier']
outcome = 'delay'

X = pd.get_dummies(delays_df[predictors])
y = delays_df['delay']

classes = list(y.cat.categories)

In [ ]:
X.head(2)

### (3) Split the data into training and test sets

In [ ]:
# split into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=1)

### (4) Define and train a Naive Bayes model

In [ ]:
# run naive Bayes
clf = MultinomialNB(alpha=0.01)
clf.fit(X_train, y_train)

### (5) Predict probabilities and class membership

In [ ]:
# predict probabilities
predProb_train = clf.predict_proba(X_train)
predProb_test = clf.predict_proba(X_test)

# predict class membership
y_test_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

In [ ]:
# Subset a specific set
df = pd.concat([pd.DataFrame({'actual': y_test, 'predicted': y_test_pred}),
                pd.DataFrame(predProb_test, index=y_test.index)], axis=1)
mask = ((X_test.carrier_DL == 1) & (X_test.dayweek_7 == 1) & (X_test.schedtime_10 == 1) & 
        (X_test.dest_LGA == 1) & (X_test.origin_DCA == 1))

print(df[mask])

### (6) Generate probability frequency tables

In [ ]:
# split the original data frame into a train and test using the same random_state
train_df, test_df = train_test_split(delays_df, test_size=0.4, random_state=1)

pd.set_option('display.precision', 4)
# probability of flight status
print(train_df['delay'].value_counts() / len(train_df))
print()

for predictor in predictors:
    # construct the frequency table
    df = train_df[['delay', predictor]]
    freqTable = df.pivot_table(index='delay', columns=predictor, aggfunc=len)

    # divide each row by the sum of the row to get conditional probabilities
    propTable = freqTable.apply(lambda x: x / sum(x), axis=1)
    print(propTable)
    print()
pd.reset_option('display.precision')

### (7) Calculate the posterior probabilities for specific cases

In [ ]:
# P(delayed | Carrier = DL, Day_Week = 7, Dep_Time = 10, Dest = LGA, Origin = DCA)
P_hat_delayed = 0.0958 * 0.1609 * 0.0307 * 0.4215 * 0.5211 * 0.1977
# P(ontime | Carrier = DL, Day_Week = 7, Dep_Time = 10, Dest = LGA, Origin = DCA)
P_hat_ontime = 0.2040 * 0.1048 * 0.0519 * 0.5779 * 0.6478 * 0.8023
print('P_hat_delayed ~ ', P_hat_delayed)
print('P_hat_ontime ~ ', P_hat_ontime)

print('P(delayed|...) = ', P_hat_delayed / (P_hat_delayed + P_hat_ontime))
print('P(ontime|...) = ', P_hat_ontime / (P_hat_delayed + P_hat_ontime))

### (8) Evaluate the model using confusion matrix and ROC curve

In [ ]:
print(clf.score(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_test_pred)

print("Confusion Matrix:")
print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()  
y_test_binary = lb.fit_transform(y_test)

fpr, tpr, thresholds = roc_curve(y_test_binary, predProb_test[:,1])  
roc_auc = auc(fpr, tpr)  

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # 대각선 (랜덤 모델)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

print(f'AUC: {roc_auc:.2f}')